# Pre Analysis

In [19]:
import os
import pyreadstat
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from io import BytesIO
import tempfile
import warnings

In [20]:
# Set the output directory

output_directory = r"C:\Users\ShindeAnk\OneDrive - Kantar\Punit Team\Training Documents\BSA Training\Automation\BSA Process_Automation\Output"
output_file = "output.xlsx"
output_path = os.path.join(output_directory, output_file)


In [21]:
# Initializing global DataFrames
data_view = pd.DataFrame()
variable_view = pd.DataFrame()

# Function to handle the file upload and read the data
def handle_file_upload(change):
    global data_view, variable_view  # Declare global variables

    # Access the uploaded file from the widget
    uploaded_file = next(iter(change['new'].values()))  # Get the uploaded file content
    if uploaded_file:
        # Convert the uploaded file to a BytesIO object
        file_content = BytesIO(uploaded_file['content'])
        
        # Write the BytesIO object to a temporary file
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(file_content.getvalue())
            temp_file_path = temp_file.name
        
        # Load the data and metadata from the temporary file
        df, meta = pyreadstat.read_sav(temp_file_path)
        
        # Function to replace values with labels
        def apply_value_labels(data, meta):
            for column_name in data.columns:
                if column_name in meta.variable_value_labels:
                    value_labels = meta.variable_value_labels[column_name]
                    data[column_name] = data[column_name].map(value_labels)
            return data
        
        # Apply the value labels to the data
        data_view = apply_value_labels(df.copy(), meta)
        
        # Variable View: metadata about the variables
        variable_view = pd.DataFrame({
            'Variable Name': meta.column_names,
            'Variable Label': meta.column_labels
        })
        
        # Display the head of data_view and variable_view
        print("Data View after applying value labels:")
        display(data_view.head())
        print("Variable View:")
        display(variable_view.head(20))

# Create the file upload widget
upload_widget = widgets.FileUpload(accept='.sav', multiple=False)

# Display the widget
display(upload_widget)

# Attach the event handler to process the file once it's uploaded
upload_widget.observe(handle_file_upload, names='value')

FileUpload(value={}, accept='.sav', description='Upload')

In [22]:
data_view.head()

""


In [23]:
data_view.shape

(0, 0)

In [24]:
variable_view.head(20)

""


In [ ]:
variable_view.shape

In [12]:
# Create a dictionary for mapping variable names to labels
mapping_dict = variable_view.set_index('Variable Name')['Variable Label'].to_dict()

In [13]:
def rename_columns(df, mapping_dict):
    return df.rename(columns=mapping_dict)

# Replace the variable names in the "UK Category" DataFrame with the corresponding labels
data_view_renamed = rename_columns(data_view, mapping_dict)

# Now df_uk_category_renamed has the columns with the actual names from the "Variable label" sheet


In [25]:
data_view_renamed.head()

,Respondent_id (KANSOS ID),Fieldwork country,PersistentRespondentId,Timings_yyyymmdd,Timings_WeekNumber,Brand,Month,Year,Month and Year,Weighting variable,...,BEAST - CM_GrpdSegs,BEAST - MarketFactors_SoEUnrealised,None,None,None,None,None,None,None,None
0,I_202212_BRA_434393844,Brazil,,20220326.0,202212.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,2.5113,...,Steady,0.000000,I_202212_BRA_4343938442022041,I_202212_BRA_434393844,434393844,202204.0,BRA,1.0,0.0,TRUE
1,I_202212_BRA_464393944,Brazil,,20220327.0,202212.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,0.8579,...,Steady,0.000000,I_202212_BRA_4643939442022041,I_202212_BRA_464393944,464393944,202204.0,BRA,1.0,1.0,TRUE
2,I_202212_BRA_464393946,Brazil,,20220327.0,202212.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,0.8579,...,Steady,0.000000,I_202212_BRA_4643939462022041,I_202212_BRA_464393946,464393946,202204.0,BRA,1.0,1.0,TRUE
3,I_202213_BRA_404084541,Brazil,,20220401.0,202213.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,1.3867,...,Steady,0.435618,I_202213_BRA_4040845412022041,I_202213_BRA_404084541,404084541,202204.0,BRA,1.0,0.0,TRUE
4,I_202213_BRA_404094948,Brazil,,20220401.0,202213.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,0.8464,...,Uncommitted,0.000000,I_202213_BRA_4040949482022041,I_202213_BRA_404094948,404094948,202204.0,BRA,1.0,0.0,TRUE


##### Apppy Column Filters

In [26]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Assuming data_view_renamed is your DataFrame (load it dynamically)
# Example DataFrame (replace this with your actual data loading code)
# data_view_renamed = pd.read_spss("your_data_file.sav")  # Replace with your actual file path

# Create a dropdown widget for selecting the column (populate options dynamically)
column_selector = widgets.Dropdown(
    options=data_view_renamed.columns,
    description='Select Column:',
    style={'description_width': 'initial'},
)

# Create a SelectMultiple widget for selecting multiple values (initially empty, to be populated dynamically)
value_selector = widgets.SelectMultiple(
    options=[],
    description='Select Values:',
    style={'description_width': 'initial'},
)

# Button to add the filter
add_filter_button = widgets.Button(description="Add Filter")
# Button to apply all filters
apply_filters_button = widgets.Button(description="Apply All Filters")

# List to store the filters
filters = []

# Function to update value dropdown based on the selected column
def update_value_dropdown(*args):
    selected_column = column_selector.value
    unique_values = data_view_renamed[selected_column].unique()
    value_selector.options = unique_values

# Update value dropdown whenever the selected column changes
column_selector.observe(update_value_dropdown, 'value')

# Function to add a filter to the list
def add_filter(button):
    selected_column = column_selector.value
    selected_values = list(value_selector.value)
    filters.append((selected_column, selected_values))
    print(f"Added filter: {selected_column} in {selected_values}")

# Function to apply all filters and display the filtered DataFrame
def apply_filters(button):
    global filtered_df
    filtered_df = data_view_renamed.copy()
    for column, values in filters:
        filtered_df = filtered_df[filtered_df[column].isin(values)]
        print(f"Column '{column}' found in DataFrame.")
    display(filtered_df.head())

# Bind the functions to the button click events
add_filter_button.on_click(add_filter)
apply_filters_button.on_click(apply_filters)

# Display the UI components
display(column_selector)
display(value_selector)
display(add_filter_button)
display(apply_filters_button)


Dropdown(description='Select Column:', options=('Respondent_id (KANSOS ID)', 'Fieldwork country', 'PersistentR…

SelectMultiple(description='Select Values:', options=(), style=DescriptionStyle(description_width='initial'), …

Button(description='Add Filter', style=ButtonStyle())

Button(description='Apply All Filters', style=ButtonStyle())

Added filter: Brand in ['Coca-Cola/Coca-Cola Classic', 'Fanta', 'Sprite']
Column 'Brand' found in DataFrame.


,Respondent_id (KANSOS ID),Fieldwork country,PersistentRespondentId,Timings_yyyymmdd,Timings_WeekNumber,Brand,Month,Year,Month and Year,Weighting variable,...,BEAST - CM_GrpdSegs,BEAST - MarketFactors_SoEUnrealised,None,None,None,None,None,None,None,None
0,I_202212_BRA_434393844,Brazil,,20220326.0,202212.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,2.5113,...,Steady,0.000000,I_202212_BRA_4343938442022041,I_202212_BRA_434393844,434393844,202204.0,BRA,1.0,0.0,TRUE
1,I_202212_BRA_464393944,Brazil,,20220327.0,202212.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,0.8579,...,Steady,0.000000,I_202212_BRA_4643939442022041,I_202212_BRA_464393944,464393944,202204.0,BRA,1.0,1.0,TRUE
2,I_202212_BRA_464393946,Brazil,,20220327.0,202212.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,0.8579,...,Steady,0.000000,I_202212_BRA_4643939462022041,I_202212_BRA_464393946,464393946,202204.0,BRA,1.0,1.0,TRUE
3,I_202213_BRA_404084541,Brazil,,20220401.0,202213.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,1.3867,...,Steady,0.435618,I_202213_BRA_4040845412022041,I_202213_BRA_404084541,404084541,202204.0,BRA,1.0,0.0,TRUE
4,I_202213_BRA_404094948,Brazil,,20220401.0,202213.0,Coca-Cola/Coca-Cola Classic,4.0,2022.0,April 2022,0.8464,...,Uncommitted,0.000000,I_202213_BRA_4040949482022041,I_202213_BRA_404094948,404094948,202204.0,BRA,1.0,0.0,TRUE


#### Converting the required imageries columns data to intergers ("Yes" to 1 "No" to 0)

In [28]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# all_columns=data_view_renamed.copy()

# Define the widgets
available_columns_widget = widgets.SelectMultiple(
    options=filtered_df.columns,
    description='Available Columns',
    disabled=False
)

selected_columns_widget = widgets.SelectMultiple(
    options=[],
    description='Selected Columns',
    disabled=False
)

# Button to add selected columns
add_button = widgets.Button(description="Add ->")

# Button to remove selected columns
remove_button = widgets.Button(description="<- Remove")

# Button to submit the selection
submit_button = widgets.Button(description="Submit")

# Functions to handle button clicks
def add_columns(b):
    selected = available_columns_widget.value
    new_selected_columns = [col for col in selected if col not in selected_columns_widget.options]
    if new_selected_columns:
        # Remove selected columns from the available columns
        available_columns_widget.options = [col for col in available_columns_widget.options if col not in selected]
        # Add to selected columns
        selected_columns_widget.options = list(selected_columns_widget.options) + new_selected_columns

def remove_columns(b):
    selected = selected_columns_widget.value
    if selected:
        # Add removed columns back to the available columns
        available_columns_widget.options = list(available_columns_widget.options) + list(selected)
        # Remove from selected columns
        selected_columns_widget.options = [col for col in selected_columns_widget.options if col not in selected]

# Function to apply all filters and display the filtered DataFrame
def submit_columns(b):
    global filtered_df
    selected_columns = list(selected_columns_widget.options)
     # Print selected columns to verify
    print("Selected columns:", selected_columns)
    
    if not selected_columns:
        print("No columns selected.")
    else:
        # Apply the conversion only to selected columns
        for column in selected_columns:
            if column in filtered_df.columns:
                filtered_df[column] = filtered_df[column].apply(lambda x: 1 if x == 'Yes' else (0 if x == 'No' else x))
            else:
                print(f"Column '{column}' not found in DataFrame.")
    
        # Display the updated DataFrame
        display(filtered_df.head())
        print("Selected columns converted")

# Assign button actions
add_button.on_click(add_columns)
remove_button.on_click(remove_columns)
submit_button.on_click(submit_columns)

# Display the UI
display(widgets.HBox([available_columns_widget, widgets.VBox([add_button, remove_button]), selected_columns_widget]))
display(submit_button)

Button(description='Submit', style=ButtonStyle())

Selected columns: []
No columns selected.
Selected columns: ['BrandPowerGrid - Meaningful_Factor', 'BrandPowerGrid - Difference_Factor', 'BrandPowerGrid - Salience_Factor', 'BrandPowerGrid - Power', 'BrandPowerGrid - Potential', 'BrandPowerGrid - Meaningful_Power_Weight', 'BrandPowerGrid - Difference_Power_Weight', 'BrandPowerGrid - Salience_Power_Weight', 'BrandPowerGrid - Meaningful_Premium_Weight', 'BrandPowerGrid - Difference_Premium_Weight', 'BrandPowerGrid - Salience_Premium_Weight', 'BrandPowerGrid - Price_worth', 'BrandPowerGrid - CL_', 'BrandPowerGrid - Share_Gap', 'BrandPowerGrid - Cluster', 'BrandPowerGrid - M_contributionR2', 'BrandPowerGrid - D_contributionR2', 'BrandPowerGrid - S_contributionR2', 'BrandPowerGrid - Dependent', 'BrandPowerGrid - PowerIndex', 'BrandPowerGrid - ShareDependent', 'BrandPowerGrid - Loyalty', 'BrandPowerGrid - PriceDiagnostic', 'BrandPowerGrid - KWP_VolFlow', 'BEAST_MarketShare - Secure', 'BEAST_MarketShare - Unsupported', 'BEAST_MarketShare - Un

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

#### Calculate Weighted Mean and Frequency - Weights On

In [ ]:
import os
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from openpyxl import load_workbook


# Function to clear the sheet and its formatting by deleting and recreating it
def clear_excel_sheet(output_path, sheet_name):
    if os.path.exists(output_path):
        book = load_workbook(output_path)
        # Remove all sheets
        for sheet in book.sheetnames:
            std = book[sheet]
            book.remove(std)
        # Create the Results sheet
        book.create_sheet(sheet_name)
        book.save(output_path)
    else:
        # Create a new workbook and add the Results sheet
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            writer.book.create_sheet(sheet_name)
            
            
# Define function to calculate weighted mean and save to Excel
def calculate_weighted_mean(weight_column, value_columns, output_path):
    if weight_column and value_columns:
        results = []
        for value_column in value_columns:
            mean = np.sum(filtered_df[value_column] * filtered_df[weight_column]) / np.sum(filtered_df[weight_column])
            print(f"Weighted Mean of {value_column}: {mean}")
            results.append({'Measure': value_column, 'Weighted Mean': mean})

        result_df = pd.DataFrame(results)
        
        clear_excel_sheet(output_path, 'Results')

        # Load existing workbook if it exists, otherwise create a new one
        if os.path.exists(output_path):
            with pd.ExcelWriter(output_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
                result_df.to_excel(writer, sheet_name='Results', index=False, startrow=2, header=['Measure', 'Weighted Mean'])
        else:
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                result_df.to_excel(writer, sheet_name='Results', index=False, startrow=2, header=['Measure', 'Weighted Mean'])
        
        print(f"Results have been saved to '{output_path}' in the 'Results' sheet.")
    else:
        print("Please select both weight and value columns.")

# Define function to calculate weighted frequency and save to Excel
def calculate_weighted_frequency(weight_column, category_columns, output_path):
    if weight_column and category_columns:
        # Load existing workbook and determine the start row
        if os.path.exists(output_path):
            book = load_workbook(output_path)
            sheet = book['Results']
            startrow = sheet.max_row + 2
        else:
            startrow = 5

        # Write each category's weighted frequency
        with pd.ExcelWriter(output_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            for category_column in category_columns:
                weighted_counts = filtered_df.groupby(category_column).apply(lambda x: np.sum(x[weight_column])).reset_index(name='Weighted Count')
                print(f"Weighted Frequencies for {category_column}:")
                print(weighted_counts)

                weighted_counts.to_excel(writer, sheet_name='Results', index=False, startrow=startrow, header=[category_column, 'Weighted Frequency'])
                startrow += len(weighted_counts) + 2  # Adjust startrow for the next calculation
        
        print(f"Results have been saved to '{output_path}' in the 'Results' sheet.")
    else:
        print("Please select both weight and category columns.")
        
# Create dropdown widgets for selecting columns
weight_column_Dropdown = widgets.Dropdown(
    options=[(col, col) for col in filtered_df.columns],
    description='Weight Column:',
)

value_column_SelectMultiple = widgets.SelectMultiple(
    options=[(col, col) for col in filtered_df.columns if col != weight_column_Dropdown.value],
    description='Value Column:',
)

category_column_SelectMultiple = widgets.SelectMultiple(
    options=[(col, col) for col in filtered_df.columns if col != weight_column_Dropdown.value],
    description='Category Column:',
)

# Create buttons to calculate weighted mean and frequency
calculate_mean_button = widgets.Button(description="Calculate Weighted Mean")
calculate_frequency_button = widgets.Button(description="Calculate Weighted Frequency")

# Define button click handlers
def on_calculate_mean_button_clicked(b):
    calculate_weighted_mean(weight_column_Dropdown.value, value_column_SelectMultiple.value, output_path)

def on_calculate_frequency_button_clicked(b):
    calculate_weighted_frequency(weight_column_Dropdown.value, category_column_SelectMultiple.value, output_path)

# Link buttons to handlers
calculate_mean_button.on_click(on_calculate_mean_button_clicked)
calculate_frequency_button.on_click(on_calculate_frequency_button_clicked)

# Display widgets
display(weight_column_Dropdown)
display(value_column_SelectMultiple)
display(calculate_mean_button)
display(category_column_SelectMultiple)
display(calculate_frequency_button)


In [ ]:
filtered_df.shape

In [ ]:
filtered_df.head()

In [ ]:
filtered_df['Awareness'].nunique()

In [ ]:
# #Remove null value
filtered_df_clean = filtered_df.dropna()


In [ ]:
filtered_df_clean.shape

## Split file by Brand

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Assuming `filtered_df_clean` is your DataFrame
# Create a multi-select dropdown for selecting the columns to sort by
sort_multi_select = widgets.SelectMultiple(
    options=filtered_df_clean.columns,  # Get the columns from the DataFrame
    description='Sort by:',
    rows=6
)

# Create a checkbox to select ascending or descending sorting
sort_order_checkbox = widgets.Checkbox(
    value=True,
    description='Sort Ascending'
)

# Create a button to trigger the sorting
sort_button = widgets.Button(
    description="Sort DataFrame",
    button_style="primary",  # Color the button
    icon='check'  # Add a check icon to the button
)

# Variable to store the sorted DataFrame
sorted_df = None

# Function to sort the DataFrame based on the selected columns
def sort_dataframe(b):
    global sorted_df
    columns = sort_multi_select.value  # Get selected columns
    ascending = sort_order_checkbox.value  # Get sort order
    
    if columns:  # If columns are selected
        sorted_df = filtered_df_clean.sort_values(by=list(columns), ascending=ascending)
        display(sorted_df)  # Display the sorted DataFrame
        print("DataFrame sorted and saved.")
    else:
        print("Please select at least one column to sort by.")

# Link the button to the sorting function
sort_button.on_click(sort_dataframe)

# Create a UI layout
ui = widgets.VBox([
    widgets.Label("Sort the DataFrame by multiple columns:"),
    sort_multi_select,
    sort_order_checkbox,
    sort_button
])

# Display the UI
display(ui)


In [ ]:
sorted_df

##### Step3: Prepare endorsements for all the variables of  selection - Please select weights to prepare weighted endorsements

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Create widgets for column selection
all_columns = sorted_df.columns.tolist()
selected_columns = []

# Define the widgets
available_columns_widget = widgets.SelectMultiple(
    options=all_columns,
    description='Available Columns',
    disabled=False
)

selected_columns_widget = widgets.SelectMultiple(
    options=selected_columns,
    description='Selected Columns',
    disabled=False
)

# Dropdown for selecting weight column (optional)
weight_column_dropdown = widgets.Dropdown(
    options=[None] + all_columns,  # Allow "None" as an option
    description='Weight Column:',
    value=None  # Default to None
)

# Button to add selected columns
add_button = widgets.Button(description="Add ->")

# Button to remove selected columns
remove_button = widgets.Button(description="<- Remove")

# Button to submit the selection
submit_button = widgets.Button(description="Submit")

# Functions to handle button clicks
def add_columns(b):
    selected = available_columns_widget.value
    new_selected_columns = [col for col in selected if col not in selected_columns_widget.options]
    if new_selected_columns:
        # Remove selected columns from the available columns
        available_columns_widget.options = [col for col in available_columns_widget.options if col not in selected]
        # Add to selected columns
        selected_columns_widget.options = list(selected_columns_widget.options) + new_selected_columns

def remove_columns(b):
    selected = selected_columns_widget.value
    if selected:
        # Add removed columns back to the available columns
        available_columns_widget.options = list(available_columns_widget.options) + list(selected)
        # Remove from selected columns
        selected_columns_widget.options = [col for col in selected_columns_widget.options if col not in selected]
def submit_columns(b):
    if selected_columns_widget.options:
        selected_cols = list(selected_columns_widget.options)
        weight_col = weight_column_dropdown.value
        
        # Filter the DataFrame to only include numeric columns
        numeric_cols = sorted_df[selected_cols].select_dtypes(include='number')
        if weight_col:
            # Ensure that the weight column is also numeric
            if pd.api.types.is_numeric_dtype(sorted_df[weight_col]):
                mean_values_by_brand = sorted_df.groupby('Brand').apply(
                    lambda x: (numeric_cols.loc[x.index].multiply(x[weight_col], axis=0).sum() / x[weight_col].sum())
                )
            else:
                print(f"Weight column '{weight_col}' is not numeric. Please select a valid numeric column.")
                return
        else:
            mean_values_by_brand = sorted_df.groupby('Brand')[numeric_cols.columns].mean(numeric_only=True)

        # Convert mean values to percentages without rounding
        mean_values_by_brand_percentage = (mean_values_by_brand * 100)

        # Reformat the DataFrame to place the "Brand" values above the means
        mean_values_transposed = mean_values_by_brand_percentage.T
        mean_values_transposed.columns.name = None  # Remove the name from the columns
        
        # Rename the index (left-side variable) to "ENDORSEMENTS"
        mean_values_transposed.index.name = "ENDORSEMENTS"
        
        # Calculate the base (count of each brand)
        base_counts = sorted_df.groupby('Brand').size()
        
        # Create a new DataFrame with the "BASE" row
        base_row = pd.DataFrame(base_counts).T
        base_row.index = ['BASE']
        
        # Combine the "BASE" row with the transposed mean values DataFrame
        result_df = pd.concat([base_row, mean_values_transposed])
        
        # Format the mean values as percentages without rounding
        result_df.iloc[1:] = result_df.iloc[1:].applymap(lambda x: f'{x:.2f}%' if isinstance(x, float) else x)
        
        # Display the DataFrame
        display(result_df)
        
        if os.path.exists(output_path):
            # Load existing sheets
            existing_sheets = pd.ExcelFile(output_path).sheet_names
            with pd.ExcelWriter(output_path, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
                # Write the new DataFrame to a new sheet
                result_df.to_excel(writer, sheet_name='Endorsement', index=True, startrow=8)
        else:
            # Create a new file and write the DataFrame
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                result_df.to_excel(writer, sheet_name='Endorsement', index=True, startrow=8)
        
        print(f"Results have been saved to '{output_path}' in the 'Endorsement' sheet, starting from row 9.")
    else:
        print("No columns selected. Please select at least one column.")


# Assign button actions
add_button.on_click(add_columns)
remove_button.on_click(remove_columns)
submit_button.on_click(submit_columns)

# Display the UI
display(widgets.HBox([available_columns_widget, widgets.VBox([add_button, remove_button]), selected_columns_widget]))
display(weight_column_dropdown)  # Add weight column dropdown
display(submit_button)


##### Step4 correaltion for any brands you selct from filter and get that in tab of excel sheet. (all this for aware =1) - Please select weights to calculate correlation based on weights

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font

warnings.filterwarnings("ignore")


# Create widgets for brand and column selection
brands = sorted_df['Brand'].unique().tolist()
all_columns = sorted_df.columns.tolist()
selected_columns = []

# Dropdown for selecting weight column (optional)
weight_column_dropdown = widgets.Dropdown(
    options=[None] + all_columns,  # Allow "None" as an option
    description='Weight Column:',
    value=None  # Default to None
)

brand_widget = widgets.SelectMultiple(
    options=brands,
    description='Brands',
    disabled=False
)

available_columns_widget = widgets.SelectMultiple(
    options=all_columns,
    description='Available Columns',
    disabled=False
)

selected_columns_widget = widgets.SelectMultiple(
    options=selected_columns,
    description='Selected Columns',
    disabled=False
)

# Button to add selected columns
add_button = widgets.Button(description="Add ->")

# Button to remove selected columns
remove_button = widgets.Button(description="<- Remove")

# Button to submit the selection
submit_button = widgets.Button(description="Submit")

# Functions to handle button clicks
def add_columns(b):
    selected = available_columns_widget.value
    new_selected_columns = [col for col in selected if col not in selected_columns_widget.options]
    if new_selected_columns:
        available_columns_widget.options = [col for col in available_columns_widget.options if col not in selected]
        selected_columns_widget.options = list(selected_columns_widget.options) + new_selected_columns

def remove_columns(b):
    selected = selected_columns_widget.value
    if selected:
        available_columns_widget.options = list(available_columns_widget.options) + list(selected)
        selected_columns_widget.options = [col for col in selected_columns_widget.options if col not in selected]

def submit_columns(b):
    if selected_columns_widget.options and brand_widget.value:
        selected_cols = list(selected_columns_widget.options)
        selected_brands = list(brand_widget.value)
        weight_col = weight_column_dropdown.value  # Get the selected weight column
        
        # Filter the DataFrame for the selected brands
        filtered_df_2 = sorted_df[sorted_df['Brand'].isin(selected_brands)]
        
        # Ensure the selected columns contain only numeric data
        numeric_cols = filtered_df_2[selected_cols].select_dtypes(include='number')
        
        if weight_col:
            # Ensure the weight column is numeric
            if pd.api.types.is_numeric_dtype(filtered_df_2[weight_col]):
                # Apply weights to calculate weighted correlation
                weighted_df = numeric_cols.multiply(filtered_df_2[weight_col], axis=0)
                corr_matrix = weighted_df.corr()
                # Add '_weight' suffix to the sheet name if a weight column is selected
                suffix = "_weight"
            else:
                print(f"Weight column '{weight_col}' is not numeric. Please select a valid numeric column.")
                return
        else:
            # Simple correlation without weights
            corr_matrix = numeric_cols.corr()
            suffix = ""  # No suffix if no weight column is selected
        
        # Format the correlation matrix to 2 decimal places
        corr_matrix = corr_matrix.round(3)
        
        # Prepare the sheet name using the selected brand names
        brand_names_str = "_".join(map(str, selected_brands))
        sheet_name = f'Correlation_{brand_names_str}{suffix}'
        
        if os.path.exists(output_path):
            # Load the existing workbook
            book = load_workbook(output_path)
            # Check if the sheet already exists
            if sheet_name in book.sheetnames:
                # If the sheet exists, delete it first and save the workbook
                del book[sheet_name]
                book.save(output_path)  # Save after deletion to update the workbook
                print(f"Sheet '{sheet_name}' already exists. Replacing with the new data.")
            else:
                print(f"Creating new sheet '{sheet_name}'.")
            
            # Reopen the workbook to append the new sheet with updated data
            with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
                writer.book = load_workbook(output_path)  # Reopen the workbook
                corr_matrix.to_excel(writer, sheet_name=sheet_name, index=True, startrow=8)
        else:
            # Create a new workbook and write to it
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                corr_matrix.to_excel(writer, sheet_name=sheet_name, index=True, startrow=8)
        
        # Load the workbook to apply conditional formatting
        book = load_workbook(output_path)
        sheet = book[sheet_name]
        
        # Define the styles
        sky_blue_fill = PatternFill(start_color="87CEEB", end_color="87CEEB", fill_type="solid")
        black_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
        white_font = Font(color="FFFFFF")

        # Apply the formatting
        for row in sheet.iter_rows(min_row=9, min_col=2, max_col=sheet.max_column, max_row=sheet.max_row):
            for cell in row:
                if isinstance(cell.value, (int, float)):
                    if cell.value > 0.5 and cell.value < 1:
                        cell.fill = sky_blue_fill
                    elif cell.value == 1:
                        cell.fill = black_fill
                        cell.font = white_font

        # Save the workbook with the formatting
        book.save(output_path)
        print("Correlation matrix calculations are done and saved!")
    else:
        print("Please select at least one brand and one column.")

# Assign button actions
add_button.on_click(add_columns)
remove_button.on_click(remove_columns)
submit_button.on_click(submit_columns)

# Display the UI
display(widgets.HBox([brand_widget, available_columns_widget, widgets.VBox([add_button, remove_button]), selected_columns_widget]))
display(weight_column_dropdown)  # Add weight column dropdown
display(submit_button)


#### >>>>>>>>>>>>>>FACTOR ANALYSIS<<<<<<<<<<<<<<<<<<<
<!-- Weights off - Yet to implement weight case -->

In [ ]:
# !pip install factor-analyzer --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [ ]:
# filtered_df.head()
# # filtered_df.info()

In [ ]:
import os
import pandas as pd
from ipywidgets import widgets, HBox, VBox, Label
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# Assuming filtered_df is your DataFrame
all_columns = sorted_df.columns.tolist()
selected_columns = []

# Extract unique brands
brands = sorted_df['Brand'].unique().tolist()

# Define the widgets
available_columns_widget = widgets.SelectMultiple(
    options=all_columns,
    description='Available Columns',
    disabled=False
)

selected_columns_widget = widgets.SelectMultiple(
    options=selected_columns,
    description='Selected Columns',
    disabled=False
)

num_factors_widget = widgets.IntText(
    description='Number of Factors:',
    disabled=False
)

# Multi-selector for brands
brands_widget = widgets.SelectMultiple(
    options=brands,
    description='Select Brands',
    disabled=False
)

# Button to add selected columns
add_button = widgets.Button(description="Add ->")

# Button to remove selected columns
remove_button = widgets.Button(description="<- Remove")

# Button to submit the selection
submit_button = widgets.Button(description="Submit")

# Functions to handle button clicks
def add_columns(b):
    selected = available_columns_widget.value
    new_selected_columns = [col for col in selected if col not in selected_columns_widget.options]
    if new_selected_columns:
        available_columns_widget.options = [col for col in available_columns_widget.options if col not in selected]
        selected_columns_widget.options = list(selected_columns_widget.options) + new_selected_columns

def remove_columns(b):
    selected = selected_columns_widget.value
    if selected:
        available_columns_widget.options = list(available_columns_widget.options) + list(selected)
        selected_columns_widget.options = [col for col in selected_columns_widget.options if col not in selected]

def submit_columns(b):
    global analysis_df 
    if selected_columns_widget.options and num_factors_widget.value:
        selected_cols = list(selected_columns_widget.options)
        selected_brands = list(brands_widget.value)
        print(f"Selected brands is : {selected_brands}")
        
        # Filter DataFrame based on selected brands
        filtered_df_brands = sorted_df[sorted_df['Brand'].isin(selected_brands)]
        analysis_df = filtered_df_brands[selected_cols]
        print(analysis_df.shape)
        
        try:
            # Perform factor analysis
            num_factors = num_factors_widget.value
            if num_factors > len(selected_cols):
                raise ValueError(f"Number of factors ({num_factors}) cannot exceed the number of selected columns ({len(selected_cols)}).")
            print(f"Selected Columns: {selected_cols}")
            print(f"Filtered DataFrame Shape: {analysis_df.shape}")
            print(f"Number of Factors: {num_factors}")
            
            # Perform factor analysis
            cov_analysis_df = analysis_df.cov() 
            fa = FactorAnalyzer(n_factors=num_factors, method='principal',rotation="varimax")
            fa.fit(cov_analysis_df) # fit analysis_df only for correlation factor analysis
            
            ev, v = fa.get_eigenvalues()
            print(ev)
            plt.scatter(range(1,analysis_df.shape[1]+1),ev)
            plt.plot(range(1,analysis_df.shape[1]+1),ev)
            plt.title('Scree Plot')
            plt.xlabel('Factors')
            plt.ylabel('Eigenvalue')
            plt.grid()
            plt.show()
            
            # Get the factor loadings
            loadings = fa.loadings_
            loadings_df = pd.DataFrame(loadings, index=selected_cols, columns=[f'Factor {i+1}' for i in range(num_factors)])
            print(loadings_df)

            sheet_name = 'Factor Analysis'
            
            if os.path.exists(output_path):
                # Load the workbook using openpyxl
                book = load_workbook(output_path)
                with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
                    writer.book = book
                    # Remove the existing sheet if it exists
                    if sheet_name in book.sheetnames:
                        del book[sheet_name]
                    # Write loadings to Excel
                    loadings_df.to_excel(writer, sheet_name=sheet_name)
            else:
                # Create a new workbook and save the results
                with pd.ExcelWriter(output_path, engine='openpyxl', mode='w') as writer:
                    loadings_df.to_excel(writer, sheet_name=sheet_name)

            print(f"Factor loadings saved to '{output_file}' in the '{sheet_name}' sheet.")
        
        except Exception as e:
            print(f"Error during factor analysis: {e}")
    else:
        print("Please select at least one column, specify the number of factors, and select at least one brand.")

# Assign button actions
add_button.on_click(add_columns)
remove_button.on_click(remove_columns)
submit_button.on_click(submit_columns)

# Display the UI
display(HBox([available_columns_widget, VBox([add_button, remove_button]), selected_columns_widget]))
display(num_factors_widget)  # Display the IntText widget for number of factors
display(brands_widget)  # Display the multi-selector for brands
display(submit_button)


# >>>>>>>>>>>>>>>>>>>>> Regression<<<<<<<<<<<<<<<<<<<<<<<<<<

In [ ]:
# >>>>>>> Enter and stepwise regression<<<<<<<<<

import os
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.preprocessing import StandardScaler

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Your code here


# Assuming `filtered_df` is your input DataFrame
columns = filtered_df.columns.tolist()

# Extract unique brands from the 'Brand' column
brands = filtered_df['Brand'].unique().tolist()

# Create multi-select widgets for independent and dependent variables
dependent_selector = widgets.Dropdown(
    options=columns,
    description='Dependent',
    disabled=False
)

independent_selector = widgets.SelectMultiple(
    options=columns,
    description='Independent',
    disabled=False
)

# Create a dropdown to select regression method
method_selector = widgets.Dropdown(
    options=['Enter', 'Stepwise'],
    description='Method',
    disabled=False
)

# for multi-brand selection, use SelectMultiple
multi_brand_selector = widgets.SelectMultiple(
    options=brands,
    description='Select Brands',
    disabled=False
)

# Button to trigger the regression
button = widgets.Button(description="Run Regression")
output = widgets.Output()

# Display the selectors and the button
display(multi_brand_selector,dependent_selector, independent_selector, method_selector, button, output)

def filter_by_brand(df, selected_brands):
    # Filter the DataFrame for the selected brand(s)
    return df[df['Brand'].isin(selected_brands)]

# Function to calculate VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [sm.OLS(X[col], sm.add_constant(X.drop(col, axis=1))).fit().rsquared for col in X.columns]
    vif_data["VIF"] = 1 / (1 - vif_data["VIF"])
    return vif_data

# Function to perform stepwise regression
# Updated stepwise_regression function with saving functionality
def stepwise_regression(X, y, initial_list=[], threshold_in=0.05, threshold_out=0.10):
    included = list(initial_list)
    stepwise_results = []
    step = 1
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    # Ensure output_path is provided for saving results
    if output_path is None:
        raise ValueError("Output path must be specified for saving results.")
    
    while True:
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        
        for new_column in excluded:
            X_temp = X[included + [new_column]]
            scaler = StandardScaler()
            X_temp_scaled = scaler.fit_transform(X_temp)
            X_temp_scaled_df = pd.DataFrame(X_temp_scaled, columns=included + [new_column])
            model = sm.OLS(y, sm.add_constant(X_temp_scaled_df)).fit()
            new_pval[new_column] = model.pvalues[new_column]
        
        best_pval = new_pval.min()
        
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            X_included = X[included]
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_included)
            X_scaled_df = pd.DataFrame(X_scaled, columns=included)
            model = sm.OLS(y, sm.add_constant(X_scaled_df)).fit()

            coefficients = model.params[1:]
            standard_errors = model.bse[1:]
            t_stats = model.tvalues[1:]
            p_values = model.pvalues[1:]
            standardized_coefficients = coefficients * (X_scaled_df.std() / y.std())
            vif_df = calculate_vif(X_scaled_df)
            
            output_df = pd.DataFrame({
                
                'Standardized Coefficients (Beta)': standardized_coefficients,
                'Sig. (p-value)': p_values
                
            })
            
            stepwise_results.append((f'Step {step} - Add {best_feature}', output_df))
            step += 1
            
        
        X_included = X[included]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_included)
        X_scaled_df = pd.DataFrame(X_scaled, columns=included)
        model = sm.OLS(y, sm.add_constant(X_scaled_df)).fit()
        pvalues = model.pvalues[1:]

        worst_pval = pvalues.max()
        
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            X_included = X[included]
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_included)
            X_scaled_df = pd.DataFrame(X_scaled, columns=included)
            model = sm.OLS(y, sm.add_constant(X_scaled_df)).fit()

            coefficients = model.params[1:]
            standard_errors = model.bse[1:]
            t_stats = model.tvalues[1:]
            p_values = model.pvalues[1:]
            standardized_coefficients = coefficients * (X_scaled_df.std() / y.std())
            vif_df = calculate_vif(X_scaled_df)
            
            output_df = pd.DataFrame({
                
                'Standardized Coefficients (Beta)': standardized_coefficients,
                'Sig. (p-value)': p_values
            })
            
            stepwise_results.append((f'Step {step} - Remove {worst_feature}', output_df))
            step += 1
        
        if best_pval >= threshold_in and worst_pval <= threshold_out:
            break
    
    return stepwise_results, included

def get_unique_sheet_name(base_name, workbook):
    sheet_name= base_name
    counter=1
    while sheet_name in workbook.sheetnames:
        sheet_name=f"{base_name}{counter}"
        counter+=1
    return sheet_name


# Function to perform linear regression based on selections
def run_regression(button):
    global filtered_by_brand_df
    with output:
        clear_output()  # Clear previous output
        
        # Filter the DataFrame by selected brand(s)
        selected_brands = list(multi_brand_selector.value) 
        if not selected_brands:
            print("Please select at least one brand!")
            return
        
        filtered_by_brand_df = filter_by_brand(filtered_df, selected_brands)
        
        dependent_var = dependent_selector.value
        independent_vars = list(independent_selector.value)
        method = method_selector.value
        
        if not independent_vars or not dependent_var:
            print("Please select both dependent and independent variables!")
            return

        X = filtered_by_brand_df[independent_vars]
        y = filtered_by_brand_df[dependent_var]

        X_with_const = sm.add_constant(X)

        if method == 'Enter':
            # Perform Enter regression
            model = sm.OLS(y, X_with_const).fit()

            # Extract coefficients and metrics
            coefficients = model.params
            standard_errors = model.bse
            t_stats = model.tvalues
            p_values = model.pvalues
            standardized_coefficients = coefficients * (X_with_const.std() / y.std())

            # Create DataFrame for the Enter results
            enter_result_df = pd.DataFrame({
                'Unstandardized Coefficients (B)': coefficients,
                'Standardized Coefficients (Beta)': standardized_coefficients,
                'Standard Error': standard_errors,
                't': t_stats,
                'Sig. (p-value)': p_values
            })

            # Write Enter regression results to Excel
            with pd.ExcelWriter(output_path, engine='openpyxl', mode='a' if os.path.exists(output_path) else 'w') as writer:
                workbook = writer.book
                sheet_name = get_unique_sheet_name('Enter Results', workbook)  # Get unique sheet name
                enter_result_df.to_excel(writer, sheet_name=sheet_name, index=True)

            print("Enter regression completed and saved.")

        elif method == 'Stepwise':
            stepwise_results, final_vars = stepwise_regression(X, y)
            # Load the workbook if it exists, or initialize it
            if os.path.exists(output_path):
                workbook = load_workbook(output_path)
                sheet_name = get_unique_sheet_name('Stepwise_Results', workbook)
                if sheet_name in workbook.sheetnames:
                    existing_data = pd.read_excel(output_path, sheet_name=sheet_name)
                else:
                    existing_data = pd.DataFrame()
            else:
                workbook = None
                sheet_name = 'Stepwise_Results'
                existing_data = pd.DataFrame()

            try:
                # Open the writer and write the data inside the with block
                with pd.ExcelWriter(output_path, engine='openpyxl', mode='a' if os.path.exists(output_path) else 'w', if_sheet_exists="overlay") as writer:
                    if workbook:
                        writer.book = workbook

                    step_row = len(existing_data)  # Start after the existing data

                    for step_name, result_df in stepwise_results:
                        # Add a header for each step
                        header_df = pd.DataFrame({step_name: ['']})
                        header_df.to_excel(writer, sheet_name=sheet_name, startrow=step_row, index=False, header=False)
                        step_row += 1

                        # Write stepwise result DataFrame
                        result_df.to_excel(writer, sheet_name=sheet_name, startrow=step_row, index=True)
                        step_row += len(result_df) + 2  # Increment row for next step

                    print(f"Stepwise regression results saved to sheet: {sheet_name}")

            except Exception as e:
                print(f"An error occurred while saving results: {e}")
            

# Attach the run_regression function to the button click
button.on_click(run_regression)
